In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.signal import stft
import os
import pandas as pd
import sys
import glob
from scipy.stats import norm

In [3]:
class SignalProcessor:
    def __init__(self, flow, sampling_rate):
        self.flow = flow
        self.sampling_rate = sampling_rate
        self.downsampled_signals = self.downsample_signals()
        self.delta_seqs = self.calculate_delta_sequences()

    def downsample_signals(self):
        downsampled_signals = []
        for i in range(self.flow.shape[0]):
            signal = self.flow[i] 

            # Downsample the signal to the specified sampling rate
            downsampled_signal = signal[::self.sampling_rate]

            # Append the downsampled signal to list
            downsampled_signals.append(downsampled_signal)

        return downsampled_signals

    def calculate_delta_sequences(self):
        delta_seqs = []
        for signal in self.downsampled_signals:
            previous = signal[0]
            delta_seq = []
            for current in signal[1:]:
                delta = current - previous
                delta_seq.append(delta)
                previous = current
            delta_seqs.append(delta_seq)

        return delta_seqs


In [4]:
class GaussianQuantizer:
    def __init__(self, data, num_bins, alphabet):
        self.data = data
        self.num_bins = num_bins
        self.alphabet = alphabet
        self.mean, self.std = norm.fit(data)
        self.bin_edges = self.calculate_bin_edges()

    def calculate_bin_edges(self):
        hist_values, bin_edges = np.histogram(self.data, bins=self.num_bins, density=True)
        return bin_edges

    def value_to_alphabet(self, value):
        for i in range(len(self.bin_edges) - 1):
            if self.bin_edges[i] <= value < self.bin_edges[i+1]:
                return self.alphabet[i]
        return self.alphabet[-1]

    def convert_dataset(self, dataset):
        converted_dataset = []
        for row in dataset:
            converted_row = [self.value_to_alphabet(value) for value in row]
            converted_dataset.append(converted_row)
        return converted_dataset

In [2]:
class DatasetDic1:
    def __init__(self, f, p, v):
        self.f = f
        self.p = p
        self.v = v
        self.dataset = self.create_dataset()
        self.words_dataset = self.create_words_dataset()
        self.word_to_index = self.create_word_to_index()

    def create_dataset(self):
        concatenated_samples = []
        for i in range(self.v.shape[0]):
            concatenated_sample = np.concatenate([self.f[i], self.p[i], self.v[i]])
            concatenated_samples.append(concatenated_sample)

        # Convert the list of concatenated samples to a numpy
        dataset = np.array(concatenated_samples)
        return dataset

    def create_words_dataset(self):
        words_dataset = []

        # Iterate through the samples and create words of length 3
        for sample in self.dataset:
            words = [''.join(sample[i:i+3]) for i in range(0, len(sample), 3)]
            words_dataset.append(words)
        return words_dataset

    def create_word_to_index(self):
        unique_dataset_words = np.unique(self.words_dataset)
        word_to_index = {word: index for index, word in enumerate(unique_dataset_words)}

        mapping_function = lambda x: word_to_index[x] if x in word_to_index else x
        mapped_dataset = np.vectorize(mapping_function)(self.words_dataset)
        
        return word_to_index, mapped_dataset


In [8]:
class downsamplePlotter:
    def __init__(self, pressure_signal,downsampling_rate):
        self.pressure_signal = pressure_signal
        self.downsampling_rate = downsampling_rate
        self.x_coord = np.arange(0, pressure_signal.size, self.downsampling_rate)

    def plot_pressure_signal(self):
        pressure_0 = self.pressure_signal
        fig, (ax1, ax3) = plt.subplots(nrows=2,figsize=(10, 8))

        ax1.set_title('Original Pressure Signal')
        ax1.plot(pressure_0,label='Original Signal')
        ax1.set_ylabel('Pressure')
        ax1.set_xlabel('Time')

        #ax2.set_title('Sampled signal')
        #ax2.plot(pressure_0)
        #for x in np.nditer(self.x_coord):
         #   ax2.axvline(x, color='red', linewidth=0.5)
        #ax2.set_ylabel('Pressure')
        #ax2.set_xlabel('Time')
        ax3.set_title('Downsampled Signal')
        ax3.plot(range(0, pressure_0.size, self.downsampling_rate), pressure_0[::self.downsampling_rate], 'ro', markersize=3, color="r")
        ax3.plot(range(0, pressure_0.size, self.downsampling_rate), pressure_0[::self.downsampling_rate], 'b-', linewidth=0.5, color='blue')
        ax3.set_ylabel('Pressure')
        ax3.set_xlabel('Time')

        fig.subplots_adjust(hspace=0.5)
        plt.show()

In [3]:
class gaussian_histogram:
    def __init__(self, data,num_bins,num_std):
        self.data = data
        self.num_bins = num_bins
        self.num_std = num_std

    def plot_gaussian_histogram(self):
       
        plt.hist(self.data, bins=self.num_bins, edgecolor='black', density=True)

        mean, std = norm.fit(self.data)

        x = np.linspace(mean - self.num_std*std, mean + self.num_std*std)

        y = norm.pdf(x, mean, std)
        plt.plot(x, y)

        plt.xlabel('Data')
        plt.ylabel('Frequency')
        plt.title('Gaussian Distribution - All Signals')
        #plt.savefig(output_filename, dpi=300)
        plt.show()

In [1]:
class DatasetDic2:
    def __init__(self, dataset):
        self.dataset = dataset

    def create_words_dataset(self):
        words_dataset = []
        for sample in self.dataset:
            words = [''.join(sample[i:i+3]) for i in range(0, len(sample), 3)]
            words_dataset.append(words)
        self.words_dataset = np.array(words_dataset)
        return self.words_dataset
    
    def create_word_to_index(self, start_index=0):
        unique_dataset_words = np.unique(self.words_dataset)
        word_to_index = {word: index + start_index for index, word in enumerate(unique_dataset_words)}
        mapping_function = lambda x: word_to_index[x] if x in word_to_index else x
        mapped_dataset = np.vectorize(mapping_function)(self.words_dataset)
        return word_to_index, mapped_dataset
